In [111]:
import pandas as pd
import numpy as np

In [112]:
df = pd.read_csv("./MEX/mex_distance_matrix.csv")
df = df.drop(['Unnamed: 0', 'shapeID'], axis = 1)
df.head(10)

,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B10,...,MEX-ADM2-1590546715-B41,MEX-ADM2-1590546715-B42,MEX-ADM2-1590546715-B43,MEX-ADM2-1590546715-B44,MEX-ADM2-1590546715-B45,MEX-ADM2-1590546715-B46,MEX-ADM2-1590546715-B47,MEX-ADM2-1590546715-B48,MEX-ADM2-1590546715-B49,MEX-ADM2-1590546715-B50
0,0.000000,0.000000,0.055894,0.135329,0.083422,0.000000,0.000000,0.000000,0.000000,0.373148,...,3.253224,4.770442,6.130982,4.715551,4.741714,4.535519,5.693704,2.555878,4.687909,6.501927
1,0.000000,0.000000,0.056499,0.066500,0.000000,0.025508,0.000000,0.000000,0.037331,0.271604,...,3.404223,4.988287,6.347188,4.923381,4.956650,4.747555,5.924267,2.712681,4.901447,6.725514
2,0.055894,0.056499,0.000000,0.000000,0.057884,0.000000,0.134585,0.000000,0.220951,0.231698,...,3.105583,4.730229,6.085852,4.651928,4.693696,4.480813,5.690577,2.419171,4.636661,6.475511
3,0.135329,0.066500,0.000000,0.000000,0.000000,0.083889,0.086895,0.000000,0.261674,0.000000,...,3.129930,4.833049,6.178996,4.719860,4.784318,4.561692,5.826832,2.448799,4.722171,6.586940
4,0.083422,0.000000,0.057884,0.000000,0.000000,0.141160,0.000000,0.000000,0.113149,0.000000,...,3.410114,5.061998,6.415189,4.970916,5.021978,4.805364,6.027400,2.722402,4.963081,6.809557
5,0.000000,0.025508,0.000000,0.083889,0.141160,0.000000,0.056798,0.000000,0.120956,0.346029,...,3.165009,4.733489,6.090421,4.661959,4.698889,4.487869,5.686688,2.469237,4.642656,6.476130
6,0.000000,0.000000,0.134585,0.086895,0.000000,0.056798,0.000000,0.017952,0.000000,0.000000,...,3.453895,5.005463,6.365828,4.947523,4.976286,4.769213,5.931853,2.758834,4.922100,6.738951
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017952,0.000000,0.101724,0.166607,...,3.318247,4.923878,6.279360,4.844486,4.887060,4.673814,5.880633,2.625432,4.829747,6.668690
8,0.000000,0.037331,0.220951,0.261674,0.113149,0.120956,0.000000,0.101724,0.000000,0.358514,...,3.490987,4.997320,6.359982,4.951773,4.972428,4.768994,5.907632,2.794099,4.920052,6.724204
9,0.373148,0.271604,0.231698,0.000000,0.000000,0.346029,0.000000,0.166607,0.358514,0.000000,...,3.311152,5.102831,6.440214,4.962243,5.045334,4.815472,6.110443,2.638690,4.979436,6.860515


In [113]:
def get_mins(x, num, cur_id):
    
    # Delete the current polygon from the list so we do no include it in our list of closest polgyons, then cast the dict back to a list
    x_dict = dict(x)
    del x_dict[cur_id]
    x = list(x_dict.values())
    
    # sort the distances from lowest to highest
    x_sorted = sorted(x)
    
    # If there are more polgyons that are a distance of 0 than the specified number, reutrn ALL of the indices of polygons that have a distance of zero
    if x.count(0) > num:
        zeros = x_sorted[0:x.count(0)]
        x_array = np.array(x)
        return np.where(x_array == 0)[0]
    
    # Otherwise, return the indices of the num closest polygons
    else:
        minimums = x_sorted[0:num]
        x_array = np.array(x)
        indices = [list(np.where(x_array == i)[0]) for i in minimums]
        inidces = [item for sublist in indices for item in sublist]
        res = []
        [res.append(x) for x in inidces if x not in res]
        return res

In [114]:
min_list = []
num = 8
c = 0


for col, row in df.iterrows():
    
    # Get the indices of the num closest polygons to the currect one
    mins = get_mins(row, num, df.columns[c])
    
    # The get_min algo does not include the current polygon so drop that for the list of column we are going to grab from using the min indices
    ref_cols = [i for i in df.columns if i != df.columns[c]]
    
    # Get the shapeID's of the num closest polygons, append them to the overall list and increment c
    min_ids = [ref_cols[i] for i in mins]
    min_list.append(min_ids)
    c += 1
    

# Make the final data frame
min_df = pd.DataFrame(min_list)
min_df.columns = ['closest' + str(i) for i in range(0, num)]
min_df = min_df.reset_index()
min_df['index'] = df.columns
min_df = min_df.rename(columns = {'index':'shapeID'})
min_df.head()

,shapeID,closest0,closest1,closest2,closest3,closest4,closest5,closest6,closest7
0,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B11
1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4
2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B11,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B7
3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B11,MEX-ADM2-1590546715-B7
4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B9


In [115]:
min_df.shape

(50, 9)

In [117]:
min_df.to_csv("./MEX/closest_polygons.csv", index = False)